In [12]:
import requests
import base64
from pymongo import MongoClient
from dotenv import load_dotenv
import marqo as mq
import os

In [4]:
load_dotenv()

mongo_uri = os.getenv("MONGO_URI")

In [5]:
prompt = """
    You are a fashion tagging assistant. 
    Analyze the clothing item in this image and respond ONLY in JSON with the following fields:
    
    IMPORTANT: Your response must be valid JSON and nothing else. Do not include any markdown formatting or additional text.
    
    {
        "category": "",               
        "sub_category": "",           
        "primary_color": "",
        "secondary_color": "",
        "pattern": "",                
        "formality_level": 1,         
        "seasons": [],                
        "occasions": [],              
        "style_tags": [],             
        "gender_target": "",          
        "body_part": "",              
        "description": ""             
    }

    Definitions:
    - category: shirt, t-shirt, jeans, trousers, kurta, blazer, sneaker, sandal, etc.
    - sub_category: casual, formal, ethnic, sportswear.
    - primary_color: dominant visible color.
    - pattern: solid, striped, checked, floral, graphic.
    - formality_level: 1=very casual, 5=very formal.
    - seasons: list of ["summer", "winter", "monsoon", "all"].
    - occasions: ["office", "casual", "party", "date", "wedding", "travel", "festival"].
    - gender_target: menswear, womenswear, unisex.
    - body_part: upper, lower, footwear, outerwear, accessory.
    """

In [6]:
def encode_image(img_path):
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

In [34]:
# local model running using llama.cpp server (quantized model)
local_url = "http://127.0.0.1:8034/v1/chat/completions"

In [5]:
import json

def string_to_json(json_string):
    """
    Convert a JSON-formatted string to a Python dictionary.
    
    Args:
        json_string (str): A string containing JSON data
        
    Returns:
        dict: Parsed JSON data as a Python dictionary
        
    Raises:
        json.JSONDecodeError: If the string is not valid JSON
        TypeError: If the input is not a string
    """
    if not isinstance(json_string, str):
        raise TypeError("Input must be a string")
        
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        # You can customize the error message or handling as needed
        raise json.JSONDecodeError(f"Invalid JSON string: {str(e)}", e.doc, e.pos)

In [6]:
def analyze_clothing(img_path):
    img_b64 = encode_image(img_path)
    print("analyzing clothing.....")
    data = {
        "model": "Qwen3-VL-4B-Instruct-GGUF:Q4_K_M",
        "messages": [
            {
                    "role": "system",
                    "content": "You are a helpful assistant that extracts structured metadata from clothing images. Respond ONLY with valid JSON that matches the required schema."
                },
           {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{img_b64}",
                                "detail": "high"
                            }
                        }
                    ]
                }
        ],
        "logprobs": 1
    }

    response = requests.post(local_url, json=data)
    # logprobs = response.json()["choices"][0]["logprobs"]['content']
    text = response.json()["choices"][0]["message"]["content"]
    json_response = string_to_json(text)
    return json_response

In [10]:
def get_embedding(text: str):
    payload = {
        "input": text
    }
    r = requests.post(f"{local_url}/embedding", json=payload)
    data = r.json()

    # llama.cpp returns embedding in "data" → "embedding"
    return data

In [11]:
from sentence_transformers import SentenceTransformer

def get_embedding(description):
    print("Generating embedding for description: ", description)
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embedding = model.encode(description)
    return embedding

/Users/raj/Documents/GitHub/personal-stylist/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
client = MongoClient(mongo_uri)
db = client["personal-stylist"]
clothes = db["clothes_local"]

In [13]:
def save_item_to_db(id, item, img_path):
    entry = {
        "_id": id,
        "image_path": img_path,
        "category": item["category"],
        "sub_category": item["sub_category"],
        "primary_color": item["primary_color"],
        "secondary_color": item["secondary_color"],
        "pattern": item["pattern"],
        "formality_level": item["formality_level"],
        "seasons": item["seasons"],
        "occasions": item["occasions"],
        "style_tags": item["style_tags"],
        "gender_target": item["gender_target"],
        "body_part": item["body_part"],
        "description": item["description"],
    }

    clothes.insert_one(entry)
    print(f"Saved: {img_path}")

In [100]:
import marqo

mq = marqo.Client(url="http://localhost:8882")

# Create the index with minimal parameters for an unstructured index
mq.create_index(
    index_name="wardrobe-index",
    type="unstructured",
    model="hf/all-mpnet-base-v2"
)

{'acknowledged': True, 'index': 'wardrobe-index'}

In [99]:
# Delete the index
index_name = "wardrobe-index"  # Replace with your index name
try:
    mq.delete_index(index_name)
    print(f"Successfully deleted index: {index_name}")
except Exception as e:
    print(f"Error deleting index: {str(e)}")

Successfully deleted index: wardrobe-index


In [101]:
def save_to_marqo(id, description, img_path):
    doc = {
        "id": str(id),  # Ensure ID is a string
        "description": description,
        "image": f"file://{img_path}"
            }
    try:
        result = mq.index("wardrobe-index").add_documents(
            documents=[doc],
            tensor_fields=["description", "image"]  # Include both text and image fields for vectorization
        )
        print("Marqo index result:", result)
        return result
    except Exception as e:
        print(f"Error adding to Marqo: {str(e)}")
        raise

In [102]:
items = clothes.find()
for i in items:
    save_to_marqo(i['_id'], i['description'], i['image_path'])

Marqo index result: {'errors': False, 'processingTimeMs': 3087.322667997796, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': 'c5e73cf1-83f1-436e-99e6-85e49e1f43b1'}]}
Marqo index result: {'errors': False, 'processingTimeMs': 428.8131249923026, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': '5ca32b7a-7239-43c2-8238-a633f287d5be'}]}
Marqo index result: {'errors': False, 'processingTimeMs': 476.2346250063274, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': 'ffcda6d8-4b30-4f88-bfd6-dc74adedec71'}]}
Marqo index result: {'errors': False, 'processingTimeMs': 337.5436670030467, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': 'b716cee2-e84e-40c3-8dac-16b02acb9275'}]}
Marqo index result: {'errors': False, 'processingTimeMs': 291.8716249987483, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': '3d32da0d-ac84-408a-9a6c-05b4f60cc984'}]}
Marqo index result: {'errors': False, 'processingTimeMs': 274.82408299692906, 'i

In [20]:
import os
import json
import uuid

images = os.listdir("./wardrobe_images")

for image in images:
    print("Processing image: ", image)
    if image.lower().endswith(('.png', '.jpg', '.jpeg')):
        id = str(uuid.uuid4())
        img_path = os.path.join("./wardrobe_images", image)
        metadata = analyze_clothing(img_path)
        save_item_to_db(id, metadata, img_path)
        save_to_marqo(id, metadata['description'], img_path)

        

Processing image:  IMG_1142.HEIC
Processing image:  IMG_1143.HEIC
Processing image:  IMG_1143.jpg
analyzing clothing.....
Saved: ./wardrobe_images/IMG_1143.jpg
Marqo index result: {'errors': False, 'processingTimeMs': 5621.795128001395, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': '66b055b1-0516-442c-9d35-7c00a4aa91fb'}]}
Processing image:  IMG_1148.HEIC
Processing image:  IMG_1142.jpg
analyzing clothing.....
Saved: ./wardrobe_images/IMG_1142.jpg
Marqo index result: {'errors': False, 'processingTimeMs': 1740.4536260000896, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': 'c3992e1d-a0c0-4b03-bec8-54d99fac19d6'}]}
Processing image:  IMG_1140.jpg
analyzing clothing.....
Saved: ./wardrobe_images/IMG_1140.jpg
Marqo index result: {'errors': False, 'processingTimeMs': 1625.7938760008983, 'index_name': 'wardrobe-index', 'items': [{'status': 200, '_id': 'aa6eb43e-aa42-4aaa-a4f2-9398bc4008c7'}]}
Processing image:  IMG_1144.HEIC
Processing image:  IMG_1141.jpg


In [59]:
def get_style_candidates(query):
    results = mq.index("wardrobe-index").search(
        q=query,
        searchable_attributes=["description"],
        limit=20
    )
    return results["hits"]

In [60]:
query = "A minimal semi-formal outfit for a 15°C office meeting"

In [61]:
hits = get_style_candidates(query)

In [62]:
len(hits)

12

In [63]:
items = [clothes.find_one({"_id": hit["id"]}) for hit in hits]

In [64]:
len(items)

12

In [69]:
items

[None,
 {'_id': '8f73978e-6056-416d-8172-56b7d65d5a77',
  'image_path': './wardrobe_images/IMG_1150.jpg',
  'category': 'trousers',
  'sub_category': 'casual',
  'primary_color': 'brown',
  'secondary_color': 'black',
  'pattern': 'solid',
  'formality_level': 1,
  'seasons': ['all'],
  'occasions': ['casual', 'travel'],
  'style_tags': ['cargo', 'jogger'],
  'gender_target': 'unisex',
  'body_part': 'lower',
  'description': 'Brown cargo jogger pants with elastic waist and drawstrings, featuring large side pockets and tapered cuffs.'},
 None,
 {'_id': '285400cf-3632-4cd5-afd3-ad150c200f71',
  'image_path': './wardrobe_images/IMG_1144.jpg',
  'category': 'shirt',
  'sub_category': 'casual',
  'primary_color': 'brown',
  'secondary_color': 'beige',
  'pattern': 'striped',
  'formality_level': 1,
  'seasons': ['summer', 'all'],
  'occasions': ['casual', 'office', 'travel'],
  'style_tags': [],
  'gender_target': 'unisex',
  'body_part': 'upper',
  'description': 'A casual polo shirt with

In [76]:
def categorize(items):
    slots = { "top": [], "bottom": [], "shoes": [], "outerwear": [] }
    for i in items:
        if i is not None:
            part = i["body_part"]
            if part == "upper":
                slots["top"].append(i)
            elif part == "lower":
                slots["bottom"].append(i)
            elif part == "footwear":
                slots["shoes"].append(i)
            elif part == "outerwear":
                slots["outerwear"].append(i)
    return slots

In [77]:
slots = categorize(items)
slots

{'top': [{'_id': '285400cf-3632-4cd5-afd3-ad150c200f71',
   'image_path': './wardrobe_images/IMG_1144.jpg',
   'category': 'shirt',
   'sub_category': 'casual',
   'primary_color': 'brown',
   'secondary_color': 'beige',
   'pattern': 'striped',
   'formality_level': 1,
   'seasons': ['summer', 'all'],
   'occasions': ['casual', 'office', 'travel'],
   'style_tags': [],
   'gender_target': 'unisex',
   'body_part': 'upper',
   'description': 'A casual polo shirt with a zippered front, featuring vertical striped patterns in brown and beige tones.'},
  {'_id': '66d0af59-f1c5-460a-9976-6bafb1858975',
   'image_path': './wardrobe_images/IMG_1145.jpg',
   'category': 't-shirt',
   'sub_category': 'casual',
   'primary_color': 'yellow',
   'secondary_color': 'black',
   'pattern': 'graphic',
   'formality_level': 1,
   'seasons': ['summer', 'all'],
   'occasions': ['casual', 'party', 'date'],
   'style_tags': ['funny', 'graphic tee'],
   'gender_target': 'unisex',
   'body_part': 'upper',
  

In [78]:
def top_n(items, n=3):
    return items[:n]

In [79]:
import itertools

def generate_candidates(slots):
    tops = top_n(slots["top"])
    bottoms = top_n(slots["bottom"])

    outfits = []
    print(tops)
    print(bottoms)

    for t, b in itertools.product(tops, bottoms):
        outfits.append({
                "top": t,
                "bottom": b,

            })

    return outfits

In [80]:
outfits = generate_candidates(slots)
outfits

[{'_id': '285400cf-3632-4cd5-afd3-ad150c200f71', 'image_path': './wardrobe_images/IMG_1144.jpg', 'category': 'shirt', 'sub_category': 'casual', 'primary_color': 'brown', 'secondary_color': 'beige', 'pattern': 'striped', 'formality_level': 1, 'seasons': ['summer', 'all'], 'occasions': ['casual', 'office', 'travel'], 'style_tags': [], 'gender_target': 'unisex', 'body_part': 'upper', 'description': 'A casual polo shirt with a zippered front, featuring vertical striped patterns in brown and beige tones.'}, {'_id': '66d0af59-f1c5-460a-9976-6bafb1858975', 'image_path': './wardrobe_images/IMG_1145.jpg', 'category': 't-shirt', 'sub_category': 'casual', 'primary_color': 'yellow', 'secondary_color': 'black', 'pattern': 'graphic', 'formality_level': 1, 'seasons': ['summer', 'all'], 'occasions': ['casual', 'party', 'date'], 'style_tags': ['funny', 'graphic tee'], 'gender_target': 'unisex', 'body_part': 'upper', 'description': 'Yellow t-shirt with Snoopy and Peanuts graphic pattern, casual and fun 

[{'top': {'_id': '285400cf-3632-4cd5-afd3-ad150c200f71',
   'image_path': './wardrobe_images/IMG_1144.jpg',
   'category': 'shirt',
   'sub_category': 'casual',
   'primary_color': 'brown',
   'secondary_color': 'beige',
   'pattern': 'striped',
   'formality_level': 1,
   'seasons': ['summer', 'all'],
   'occasions': ['casual', 'office', 'travel'],
   'style_tags': [],
   'gender_target': 'unisex',
   'body_part': 'upper',
   'description': 'A casual polo shirt with a zippered front, featuring vertical striped patterns in brown and beige tones.'},
  'bottom': {'_id': '8f73978e-6056-416d-8172-56b7d65d5a77',
   'image_path': './wardrobe_images/IMG_1150.jpg',
   'category': 'trousers',
   'sub_category': 'casual',
   'primary_color': 'brown',
   'secondary_color': 'black',
   'pattern': 'solid',
   'formality_level': 1,
   'seasons': ['all'],
   'occasions': ['casual', 'travel'],
   'style_tags': ['cargo', 'jogger'],
   'gender_target': 'unisex',
   'body_part': 'lower',
   'description'

In [81]:
def score_outfit(outfit, occasion, weather, style_pref):
    prompt = f"""
    Rate this outfit for the given scenario.

    Outfit:
    Top: {outfit['top']['description']}
    Bottom: {outfit['bottom']['description']}

    Occasion: {occasion}
    Weather: {weather}
    User Style Preference: {style_pref}

    Return ONLY JSON with:
    {{
      "color_harmony": int (1-10),
      "occasion_fit": int (1-10),
      "style_alignment": int (1-10),
      "weather_suitability": int (1-10),
      "overall_score": float,
      "reason": "short explanation"
    }}
    """

    data = {
        "model": "Qwen3-VL-4B-Instruct-GGUF:Q4_K_M",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "logprobs": 1
    }

    response = requests.post(local_url, json=data)

    text = response.json()["choices"][0]["message"]["content"]
    json_response = string_to_json(text)
    return json_response


In [87]:
weather = "sunny"
occasion = "brunch"
style_pref = "casual"


In [88]:
for outfit in outfits:
    result = score_outfit(outfit, occasion, weather, style_pref)
    print(result)
    outfit["score"] = result["overall_score"]
    outfit["reason"] = result["reason"]


{'color_harmony': 7, 'occasion_fit': 6, 'style_alignment': 9, 'weather_suitability': 8, 'overall_score': 7.2, 'reason': 'Casual and comfortable, but may be too relaxed for brunch'}
{'color_harmony': 7, 'occasion_fit': 8, 'style_alignment': 9, 'weather_suitability': 8, 'overall_score': 7.8, 'reason': 'Casual and sunny-friendly, but the color palette is somewhat muted and understated for brunch.'}
{'color_harmony': 7, 'occasion_fit': 8, 'style_alignment': 9, 'weather_suitability': 8, 'overall_score': 8.0, 'reason': 'Casual and sunny-friendly, but the dark jeans contrast with the earthy stripes, slightly limiting color harmony.'}
{'color_harmony': 7, 'occasion_fit': 8, 'style_alignment': 9, 'weather_suitability': 8, 'overall_score': 7.8, 'reason': 'Fun, casual look perfect for brunch, but yellow and brown clash slightly.'}
{'color_harmony': 7, 'occasion_fit': 8, 'style_alignment': 9, 'weather_suitability': 8, 'overall_score': 7.8, 'reason': 'Bright yellow top contrasts well with dark jean

In [84]:
outfits

[{'top': {'_id': '285400cf-3632-4cd5-afd3-ad150c200f71',
   'image_path': './wardrobe_images/IMG_1144.jpg',
   'category': 'shirt',
   'sub_category': 'casual',
   'primary_color': 'brown',
   'secondary_color': 'beige',
   'pattern': 'striped',
   'formality_level': 1,
   'seasons': ['summer', 'all'],
   'occasions': ['casual', 'office', 'travel'],
   'style_tags': [],
   'gender_target': 'unisex',
   'body_part': 'upper',
   'description': 'A casual polo shirt with a zippered front, featuring vertical striped patterns in brown and beige tones.'},
  'bottom': {'_id': '8f73978e-6056-416d-8172-56b7d65d5a77',
   'image_path': './wardrobe_images/IMG_1150.jpg',
   'category': 'trousers',
   'sub_category': 'casual',
   'primary_color': 'brown',
   'secondary_color': 'black',
   'pattern': 'solid',
   'formality_level': 1,
   'seasons': ['all'],
   'occasions': ['casual', 'travel'],
   'style_tags': ['cargo', 'jogger'],
   'gender_target': 'unisex',
   'body_part': 'lower',
   'description'

In [89]:
best_outfit = max(outfits, key=lambda x: x["score"])

In [56]:
def explain_outfit(outfit):
    prompt = f"""
    Create a friendly stylist explanation for this outfit:

    {outfit}

    Include:
    - Why it works
    - Color reasoning
    - Style reasoning
    - One optional alternative suggestion
    """

    data = {
        "model": "Qwen3-VL-4B-Instruct-GGUF:Q4_K_M",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "logprobs": 1
    }

    response = requests.post(local_url, json=data)

    text = response.json()["choices"][0]["message"]["content"]
    # json_response = string_to_json(text)
    return text


In [57]:
explain_outfit(best_outfit)

'🌞 **Stylist’s Friendly Take: “Your Sunlit Sunday Best”**\n\nHey there, fashion-forward friend! 🌞 This outfit? Pure sunshine in a comfy, chic package. You’re not just dressed — you’re *dressed to vibe*. Let’s break down why this combo is a *win* (and a 8.5/10, no pressure):\n\n---\n\n✨ **Why It Works**  \nThis isn’t just “a look” — it’s a *mood*. The waffle knit tee brings cozy, everyday comfort without sacrificing style. Paired with light-wash slim jeans, it’s like a hug from your favorite hoodie… but with more structure and less sweatpants vibes. It’s effortless, versatile, and ready for anything from a coffee date to a quick office run.\n\n---\n\n🎨 **Color Reasoning: White + Blue = Sunshine & Serenity**  \nWhite = clean, crisp, and universally flattering. It’s like a blank canvas — and you’re painting your day with *your* energy.  \nBlue jeans? Think calm, cool, and collected. A light wash? That’s the “soft denim” magic — not too heavy, not too faded, just *perfectly presentable*. T

In [90]:
best_outfit

{'top': {'_id': '285400cf-3632-4cd5-afd3-ad150c200f71',
  'image_path': './wardrobe_images/IMG_1144.jpg',
  'category': 'shirt',
  'sub_category': 'casual',
  'primary_color': 'brown',
  'secondary_color': 'beige',
  'pattern': 'striped',
  'formality_level': 1,
  'seasons': ['summer', 'all'],
  'occasions': ['casual', 'office', 'travel'],
  'style_tags': [],
  'gender_target': 'unisex',
  'body_part': 'upper',
  'description': 'A casual polo shirt with a zippered front, featuring vertical striped patterns in brown and beige tones.'},
 'bottom': {'_id': '7b8eb5d5-627e-46fb-a266-1ff85fb7dd5c',
  'image_path': './wardrobe_images/IMG_1147.jpg',
  'category': 'jeans',
  'sub_category': 'casual',
  'primary_color': 'black',
  'secondary_color': '',
  'pattern': 'solid',
  'formality_level': 1,
  'seasons': ['all'],
  'occasions': ['casual', 'office', 'travel'],
  'style_tags': [],
  'gender_target': 'unisex',
  'body_part': 'lower',
  'description': 'Black skinny jeans hanging on a white ha